In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

In [ ]:
# Load the dataset
data = pd.read_csv('../data/MachineLearningRating_v3.txt',  sep='|', low_memory=False)

In [ ]:
# Display the first few rows
data.head()

In [ ]:
data.shape

In [ ]:
data.info

In [ ]:
# Check for missing values
missing_values = data.isnull().sum()
missing_values[missing_values > 0]  # Display only columns with missing values

In [ ]:
# Drop columns with a high percentage of missing values
columns_to_drop = ['CustomValueEstimate', 'CrossBorder', 'WrittenOff', 'Rebuilt', 'Converted', 'NumberOfVehiclesInFleet', 'Bank']
data_cleaned = data.drop(columns=columns_to_drop)

In [ ]:
# Impute missing values with the mode for categorical columns
categorical_columns = ['AccountType', 'MaritalStatus', 'Gender', 'VehicleType', 'make', 'Model', 'Cylinders', 'bodytype', 'NumberOfDoors', 'NewVehicle']
for column in categorical_columns:
    data_cleaned[column] = data_cleaned[column].fillna(data_cleaned[column].mode()[0])

In [ ]:
# Drop rows with missing values in columns with very few missing data
data_cleaned.dropna(subset=['CapitalOutstanding', 'VehicleIntroDate'], inplace=True)

In [ ]:
# Impute missing values for numeric columns with the mean
numeric_columns = ['mmcode', 'cubiccapacity', 'kilowatts']
for column in numeric_columns:
    data_cleaned[column] = data_cleaned[column].fillna(data_cleaned[column].mean())

In [ ]:
# Check again for missing values
data_cleaned.isnull().sum()

In [ ]:

# Basic statistics for numerical columns
numerical_summary = data_cleaned.describe()

# Basic statistics for categorical columns (frequency counts)
categorical_columns = data_cleaned.select_dtypes(include=['object', 'category']).columns
categorical_summary = data_cleaned[categorical_columns].describe()

# Display the results
print("Numerical Summary:")
print(numerical_summary)

In [ ]:
print("\nCategorical Summary:")
print(categorical_summary)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select the numerical columns
numerical_columns = ['RegistrationYear', 'cubiccapacity', 'kilowatts', 
                     'SumInsured', 'CalculatedPremiumPerTerm', 
                     'TotalPremium', 'TotalClaims']

# Plot histograms and box plots for numerical columns
for column in numerical_columns:
    plt.figure(figsize=(12, 5))
    
    # Histogram
    plt.subplot(1, 2, 1)
    sns.histplot(data_cleaned[column], kde=True, bins=30, color='blue')
    plt.title(f'Distribution of {column}')
    
    # Box plot
    plt.subplot(1, 2, 2)
    sns.boxplot(x=data_cleaned[column], color='green')
    plt.title(f'Boxplot of {column}')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot bar charts for categorical columns with top 10 high count values
for column in categorical_columns:
    # Get top 10 categories by count
    top_10_categories = data_cleaned[column].value_counts().nlargest(10).index
    # Filter the data to only include rows with the top 10 categories
    filtered_data = data_cleaned[data_cleaned[column].isin(top_10_categories)]
    
    plt.figure(figsize=(10, 5))
    sns.countplot(data=filtered_data, x=column, color='skyblue', order=top_10_categories)
    plt.title(f'Top 10 Count of {column}')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Scatter plot between numerical variables
def plot_scatter_matrix(df, columns):
    sns.pairplot(df[columns])
    plt.show()

# Selecting a subset of numerical columns for demonstration
numeric_cols = ['SumInsured', 'TotalPremium', 'TotalClaims', 'CalculatedPremiumPerTerm']
plot_scatter_matrix(data_cleaned, numeric_cols)

In [ ]:

# Boxplot for categorical vs. numerical variables
def plot_categorical_numerical(df, categorical_col, numerical_col):
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=df, x=categorical_col, y=numerical_col)
    plt.title(f'{categorical_col} vs {numerical_col}')
    plt.xticks(rotation=45)
    plt.show()

# MaritalStatus vs TotalPremium
plot_categorical_numerical(data_cleaned, 'MaritalStatus', 'TotalPremium')

# VehicleType vs SumInsured
plot_categorical_numerical(data_cleaned, 'VehicleType', 'SumInsured')

In [ ]:

# Count plot for two categorical variables
def plot_categorical_vs_categorical(df, cat1, cat2):
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x=cat1, hue=cat2)
    plt.title(f'{cat1} vs {cat2}')
    plt.xticks(rotation=45)
    plt.show()

# LegalType vs MaritalStatus
plot_categorical_vs_categorical(data_cleaned, 'LegalType', 'MaritalStatus')

# CoverType vs AccountType
plot_categorical_vs_categorical(data_cleaned, 'CoverType', 'AccountType')

In [ ]:
# Select only numeric columns
numeric_data = data_cleaned.select_dtypes(include=[np.number])

# Correlation matrix of numerical variables
plt.figure(figsize=(12, 8))
sns.heatmap(numeric_data.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# IQR Method
def detect_outliers_iqr(df, columns):
    outliers = pd.DataFrame()
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        # Identify outliers
        column_outliers = df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]
        if not column_outliers.empty:
            outliers = pd.concat([outliers, column_outliers], axis=0)
        print(f'Outliers in {column} using IQR Method:')
        print(column_outliers[[column]].head())
    return outliers

In [ ]:
# Detect outliers using IQR Method
outliers_iqr = detect_outliers_iqr(data_cleaned, numeric_columns)

In [ ]:
# Visualizing Outliers with Box Plots
def visualize_outliers(df, columns):
    plt.figure(figsize=(16, len(columns) * 2.5))
    for i, column in enumerate(columns):
        plt.subplot(len(columns)//3 + 1, 3, i + 1)
        sns.boxplot(df[column])
        plt.title(f'Box Plot of {column}')
    plt.tight_layout()
    plt.show()

In [ ]:
# Log transformation on 'cubiccapacity' and 'kilowatts'
data_cleaned['log_cubiccapacity'] = np.log1p(data_cleaned['cubiccapacity'])  # log1p applies log(x + 1)
data_cleaned['log_kilowatts'] = np.log1p(data_cleaned['kilowatts'])

# Square root transformation on 'mmcode'
data_cleaned['sqrt_mmcode'] = np.sqrt(data_cleaned['mmcode'])

# Check the new transformed data
print(data_cleaned[['log_cubiccapacity', 'log_kilowatts', 'sqrt_mmcode']].head())

# You can visualize the transformed columns to check for improvement
plt.figure(figsize=(16, 5))

plt.subplot(1, 3, 1)
sns.boxplot(data=data_cleaned, x='log_cubiccapacity')
plt.title('Log Transformed cubiccapacity')

plt.subplot(1, 3, 2)
sns.boxplot(data=data_cleaned, x='log_kilowatts')
plt.title('Log Transformed kilowatts')

plt.subplot(1, 3, 3)
sns.boxplot(data=data_cleaned, x='sqrt_mmcode')
plt.title('Square Root Transformed mmcode')

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Applying Standard Scaling (z-score normalization)
scaler = StandardScaler()
data_cleaned[['scaled_log_cubiccapacity', 'scaled_log_kilowatts', 'scaled_sqrt_mmcode']] = scaler.fit_transform(data_cleaned[['log_cubiccapacity', 'log_kilowatts', 'sqrt_mmcode']])

In [ ]:

data_cleaned.to_csv('../data/data_cleaned.csv',index=False)